In [81]:
from keras.layers import Input, Conv2D, Activation, BatchNormalization
from keras.layers.merge import Add
from keras.layers.core import Dropout

#**kwargs=keyword argument의 줄임말(사용자가 원하는 방식대로 동작하는 함수 작성하게 해줌.)

# class ReflectionPadding2D():
#     def __init__(self, padding=(1, 1), **kwargs):
#         self.padding = tuple(padding)
#         super(ReflectionPadding2D, self).__init__(**kwargs)

#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], input_shape[1] + 2 * self.padding[0], input_shape[2] + 2 * self.padding[1], input_shape[3])

#     def call(self, input_tensor, mask=None):
#         padding_width, padding_height = self.padding
#         return pad(input_tensor, [[0,0], [padding_height, padding_height], [padding_width, padding_width], [0,0] ], 'REFLECT')


def res_block(input, filters, kernel_size=(3,3), strides=(1,1), use_dropout=False):   #Resnet 블럭
    """
   순차 API(sequential API)를 사용해 케라스 Resnet 블럭을 인스턴스화 합니다.
    
    :매개변수
      - input: 입력 텐서(tensor)
      - filters: 사용하려는 필터 수
      - kernel_size: 컨볼루션(convolution)을 위한 커널(kernel) 형태
      - strides: 컨볼루션을 위한 스트라이드(strides) 형태
      - use_dropout: 드롭아웃(dropout) 사용 여부를 결정하는 boolean 값
    
    :반환
      - 케라스 모델
    """
#     x = ReflectionPadding2D((1,1))(input)   #padding
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if use_dropout:
        x = Dropout(0.5)(x)

#     x = ReflectionPadding2D((1,1))(x)
    x = Conv2D(filters=filters,
                kernel_size=kernel_size,
                strides=strides,padding='same')(x)
    x = BatchNormalization()(x)

    # 입력과 출력 사이에 직접 연결하는 두 개의 컨볼루션 레이어(layer)
    merged = Add()([input, x])
    return merged

In [102]:
from keras.layers import Input, Activation, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.core import Lambda
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from tensorflow import pad
from tensorflow.keras.layers import Layer

ngf = 64   #generator 필터 조정
input_nc = 3  #채널갯수
output_nc = 3  #채널갯수
input_shape_generator = (152, 60, input_nc)
n_blocks_gen = 9

def generator_model():
    """생성자 구조를 생성합니다."""
    # 현재 버전 : ResNet 블럭
#     inputs = Input(shape=image_shape)   # 텐서를 반환?
    inputs = Input(shape=(28,28,3))  
# inputs = Input(shape=(784,))
# x = ReflectionPadding2D((3, 3))(inputs)
    x = Conv2D(filters=ngf, kernel_size=(7,7), padding='valid')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # 필터 수를 증가시킵니다.
    n_downsampling = 2
    for i in range(n_downsampling):
        mult = 2**i
        x = Conv2D(filters=ngf*mult*2, kernel_size=(3,3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    # 9개의 ResNet 블럭을 적용시킵니다.
    #원본 이미지의 업샘플링(upsampling)에 적용되는 9개의 ResNet 블럭(block)
    mult = 2**n_downsampling
    for i in range(n_blocks_gen):
        x = res_block(x, ngf*mult, use_dropout=True)

    # 필터 수를 3(RGB)로 줄입니다.
    for i in range(n_downsampling):
        mult = 2**(n_downsampling - i)
        x = Conv2DTranspose(filters=int(ngf * mult / 2), kernel_size=(3,3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

#     x = ReflectionPadding2D((3,3))(x)
    x = Conv2D(filters=output_nc, kernel_size=(7,7), padding='valid')(x)
    x = Activation('tanh')(x)

    # 입력에서 출력으로 직접 연결되도록 하고 [-1, 1]로 중심을 바꿉니다.
    outputs = Add()([x, inputs])
    outputs = Lambda(lambda z: z/2)(outputs)

    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model

In [103]:
from keras.layers import Input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dense, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model

ndf = 64
output_nc = 3
input_shape_discriminator = (152, 60, output_nc)


def discriminator_model():
    """식별자 구조를 생성합니다."""
    n_layers, use_sigmoid = 3, False
    inputs = Input(shape=input_shape_discriminator)

    x = Conv2D(filters=ndf, kernel_size=(4,4), strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)

    nf_mult, nf_mult_prev = 1, 1
    for n in range(n_layers):
        nf_mult_prev, nf_mult = nf_mult, min(2**n, 8)
        x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

    nf_mult_prev, nf_mult = nf_mult, min(2**n_layers, 8)
    x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(filters=1, kernel_size=(4,4), strides=1, padding='same')(x)
    if use_sigmoid:
        x = Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(1024, activation='tanh')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x, name='Discriminator')
    return model

In [104]:
from keras.layers import Input
from keras.models import Model

def generator_containing_discriminator_multiple_outputs(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_images = generator(inputs)
    outputs = discriminator(generated_images)
    model = Model(inputs=inputs, outputs=[generated_images, outputs])
    return model

In [105]:
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model

image_shape = (152, 60, 3)

def perceptual_loss(y_true, y_pred):
    vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_shape)
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

In [106]:
import keras.backend as K

def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true*y_pred)

In [107]:
#학습과정

In [108]:
# 데이터를 불러옵니다.
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
image_path='C:/Users/jmlee/Desktop/4학년 1학기/졸프/vein'
# extracted_path='C:/Users/jmlee/Desktop/4학년 1학기/졸프/정맥추출'
X_train=[]
y_train=[]
for f in os.listdir(image_path):
    if f.split('.')[1] =='bmp':
        img = cv2.imread(image_path+'/'+f)
        X_train.append(img)
        
for f in os.listdir(image_path):
    if f.split('.')[1] =='jpg':
        img = cv2.imread(image_path+'/'+f)
        y_train.append(img)
        
# 모델을 초기화합니다.
g = generator_model()
d = discriminator_model()
d_on_g = generator_containing_discriminator_multiple_outputs(g, d)

# 아담 최적화 함수를 초기화 해줍니다.
g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# 모델을 컴파일(compile)해줍니다.
d.trainable = True
d.compile(optimizer=d_opt, loss=wasserstein_loss)
d.trainable = False
loss = [perceptual_loss, wasserstein_loss]
loss_weights = [100, 1]
d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
d.trainable = True

ValueError: Operands could not be broadcast together with shapes (18, 18, 3) (28, 28, 3)